###  使用 官方的SQL链 实现 数据查询

LangChain 有一个 SQL Agent，它提供了一种比链式操作更灵活的与 SQL 数据库交互的方式。使用 SQL Agent 的主要优势包括：

它可以根据数据库的模式以及数据库的内容（比如描述一个特定的表）来回答问题。
它可以通过运行生成的查询，捕获追踪栈并正确地重新生成，来从错误中恢复。
它可以根据需要多次查询数据库以回答用户的问题。
它仅从相关表中检索的方式来节省令牌。
为了初始化agent ，我们将使用 SQLDatabaseToolkit 来创建一系列工具：

创建和执行查询
检查查询语法
检索表描述

In [12]:
from langchain.chat_models import  ChatOpenAI
from langchain.agents import initialize_agent,AgentType, Tool
from langchain.utilities import SQLDatabase
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type, List, Optional, Any
import os
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
# 获取 环境变量
#大模型apikey和url
api_key = os.getenv('api_key')
base_url = os.getenv('base_url')

#天气查询的apikey
juheAppKey = os.getenv('juheAppKey')

#数据库信息
mysql_info = os.getenv('mysql_info')
mysql_info2 = os.getenv('mysql_info2')

In [14]:
#定义大模型
online_model = ChatOpenAI(
    api_key=api_key,
    model="qwen-plus",
    base_url=base_url,
    temperature=0
)

In [15]:
#定义数据库 连接
db = SQLDatabase.from_uri(mysql_info)
db2 = SQLDatabase.from_uri(mysql_info2)

In [16]:
#查看数据库中的可用表
db2.get_usable_table_names()

['order_A', 'order_B', 'user']

In [17]:
#查看表结构
print(db2.get_table_info())


CREATE TABLE `order_A` (
	id VARCHAR(255) NOT NULL, 
	name VARCHAR(50), 
	PRIMARY KEY (id)
)DEFAULT CHARSET=utf8mb3 ENGINE=InnoDB

/*
3 rows from order_A table:
id	name
02b97c44-2d18-4004-b	Item-18
09708c1b-438a-4549-9	Item-23
0ac26b2c-bed4-41f0-8	Item-2
*/


CREATE TABLE `order_B` (
	id BIGINT NOT NULL, 
	name VARCHAR(50), 
	PRIMARY KEY (id)
)DEFAULT CHARSET=utf8mb3 ENGINE=InnoDB

/*
3 rows from order_B table:
id	name

*/


CREATE TABLE user (
	id INTEGER NOT NULL COMMENT '主键ID' AUTO_INCREMENT, 
	name VARCHAR(255) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci COMMENT '姓名', 
	password VARCHAR(255) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci COMMENT '密码', 
	sex VARCHAR(255) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci COMMENT '性别', 
	age INTEGER COMMENT '年龄', 
	phone VARCHAR(255) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci COMMENT '电话', 
	PRIMARY KEY (id)
)ROW_FORMAT=DYNAMIC ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='用户信息表' COLLATE utf8mb4_unicode_ci

/*
3 rows fro

In [18]:
#查看数据库信息
print(db.get_context())

{'table_info': '\nCREATE TABLE city_stats (\n\tcity_name VARCHAR(255), \n\tpopulation VARCHAR(255), \n\tarea VARCHAR(255)\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n3 rows from city_stats table:\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821\n*/', 'table_names': 'city_stats'}


In [19]:
#使用官方 sql 数据库工具包
from langchain_community.agent_toolkits import SQLDatabaseToolkit

tookit = SQLDatabaseToolkit(db=db, llm=online_model)

#查看工具包中的工具
tools = tookit.get_tools()
tools



[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11dad1190>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11dad1190>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11dad1190>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [22]:
tools2 = SQLDatabaseToolkit(db=db2, llm=online_model, verbose=True).get_tools()
tools2

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11daa57d0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11daa57d0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11daa57d0>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [29]:
from langchain_community.agent_toolkits import create_sql_agent

sql_agent = create_sql_agent(
    llm=online_model,
    toolkit=tookit,
    verbose=True,
    # extra_tools=tools2
)

sql_agent.run('查询城市表和用户表中的所有数据')



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input:   city_statsI should query the schema of the city_stats table to understand its structure.
Action: sql_db_schema  
Action Input: city_stats  
CREATE TABLE city_stats (
	city_name VARCHAR(255), 
	population VARCHAR(255), 
	area VARCHAR(255)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from city_stats table:
city_name	population	area
北京	20000000	400000234
上海	17689300	29867200
河北	19999944	39999821
*/I should construct a query to retrieve data from the `city_stats` table. Since the user also mentioned a user table, but it does not exist in the database as per the list of tables, I will only provide data from the `city_stats` table. I will limit the results to 10 rows to follow the guidelines.

Action: sql_db_query_checker  
Action Input: SELECT city_name, population, area FROM city_stats LIMIT 10;SELECT city_name, population, area FROM city_stats LIMIT 10;The query is cor

'Here are some data from the city_stats table:\n\n| city_name | population | area      |\n|-----------|------------|-----------|\n| 北京       | 20000000   | 400000234 |\n| 上海       | 17689300   | 29867200  |\n| 河北       | 19999944   | 39999821  |\n\nThere is no user table in the database.'